# Exercise 3

In this assignment, you will find numerical solutions to the diffusion equation. In particular, you will use an implicit method, and consider problems with both Dirichlet and Neumann boundary conditions.

**Remember**
   * You are expected to use numpy and scipy libraries where appropriate.  
   * You should run each cell in order from the top of the notebook; there is no need to repeat code between cells
   * Use the "refresh kernel" button to reset everything and start again
   * Make sure your notebook runs fully & without errors, from a fresh kernel, before submitting it

## Problem Overview

The 1D diffusion equation is :

$$\frac{\partial u}{\partial t} = k\frac{\partial^2 u}{\partial x^2}$$

You should discretize this equation onto $N_x$ space points, with separation $\Delta x = h$, and into timesteps $\Delta t = \tau$.  In the equations below, I use subscript $i$ as a space index, and superscript $n$ for time indices.

Having discretized the problem, you should use the _implicit_ finite difference equation, as discussed in lectures :

$$\frac{u_i^{n+1} - u_i^n}{\tau} = k \frac{u_{i+1}^{n+1} - 2u_i^{n+1} + u_{i-1}^{n+1}}{h^2}$$

This can be written in matrix form $u^n = M u^{n+1}$ using :

$$u_i^n = - \alpha u_{i-1}^{n+1} + (1 + 2\alpha) u_i^{n+1} - \alpha u_{i+1}^{n+1}$$

where $\alpha = \frac{k \tau}{h^2}$.

In the problems below, you are asked to solve the diffusion equation in the context of the heat equation. Here, $k$ is the thermal diffusivity, given by $k = \frac{\lambda}{\rho C}$, where $\lambda$ is the thermal conductivity, $\rho$ is the density, and $C$ is the specific heat capacity. The questions below concern an iron poker of length 50cm.  You may take the thermal conductivity of iron to be a constant 59 W/m/K, its specific heat as 450 J/kg/K, and its density as 7,900 kg/m3.  You can ignore heat loss along the length of the poker.


## Part 1 - Dirichlet Boundary Conditions

The poker is initially in equilibrium, at room temperature of 20 C. At time $t = 0$, one end is thrust into a furnace at 1000 C and the other end is held in an ice bath at 0 C. Your task is to calculate the temperature distribution along the poker as a function of time.

The fact that the ends of the rod are held at fixed temperatures of 0 C and 1000 C corresponds to a Dirichlet boundary condition.  These can be included in the implicit method as follows.

The implicit finite difference equation, above, will allow us to calculate the unknown 'internal' nodes, ie. $0 < i < (N_x-1)$.  However, the boundary nodes, $i=0, N_x$, must have fixed values $d_0, d_N$.  To fix the boundaries, we take the matrix M to be of size $(N_x-2) \times (N_x-2)$, and adding a vector term :

$$u^n = Mu^{n+1} + b$$

For $N_x = 7$ (for example), this gives :

$$M = \begin{pmatrix}
1+2\alpha & -\alpha   &           &           &           & \\
-\alpha   & 1+2\alpha & -\alpha   &           &           & \\
          & -\alpha   & 1+2\alpha & -\alpha   &           & \\
          &           & -\alpha   & 1+2\alpha & -\alpha   & \\
          &           &           & -\alpha   & 1+2\alpha & \\
\end{pmatrix}$$

$$b = \begin{pmatrix}
-\alpha d_0 \\
0 \\
0 \\
0 \\
-\alpha d_N \\
\end{pmatrix}$$

You can show this gives the required finite equation for $i=1, (N-1)$, eg. :

$$u^n_1 = - \alpha u^{n+1}_2 + (1 + 2\alpha)u^{n+1}_1 - \alpha d_0$$

First, write functions that will construct the matrix equation and boundary value term.

In [21]:
%matplotlib qt
import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from matplotlib import cm



class griddy:
    def __init__(self,x,seg,jump,d0,dN,u0,k):

        self.k=k
        self.seg=seg
        self.step=x/seg
        self.jump=jump
        self.d0 = d0
        self.dN = dN
        self.a = (k * jump) / self.step**2
        self.u = np.full((seg - 2), u0)

    def matmaker(self):
        '''initialise matrix'''
        #x = self.x -2
        mat=np.full((self.seg -2 ,self.seg - 2),(-self.a))
        mat=np.tril(np.triu(mat,-1),1)
        for i in range(0,self.seg -2):
            mat[i,i]=1 + 2*self.a
        #print(mat)
        return mat

    def boundary(self):
        #x = self.x -2
        vect=np.zeros(self.seg - 2)
        vect[0] = -self.a * self.d0
        vect[-1] = -self.a * self.dN

        return vect#np.reshape(vect, (len(vect),1))

    def uvect(self):
        return self.u
    
    def timetravel(self):
        '''transport temp distribution in time'''

        print(self.u - self.boundary())
        self.u = linalg.solve(self.matmaker(),(self.u - self.boundary())) 
    
        return self.u

segments = 100
x=0.50
k=59/(7900*450)

biggrids = griddy(x,segments,1,0,1000,20,k)



Now write a function which will transport the temperature distribution at time step $n$ to time step $n+1$. You will need to use an appropriate linear algebra routine.

In [22]:
print(biggrids.timetravel())

[ 20.          20.          20.          20.          20.
  20.          20.          20.          20.          20.
  20.          20.          20.          20.          20.
  20.          20.          20.          20.          20.
  20.          20.          20.          20.          20.
  20.          20.          20.          20.          20.
  20.          20.          20.          20.          20.
  20.          20.          20.          20.          20.
  20.          20.          20.          20.          20.
  20.          20.          20.          20.          20.
  20.          20.          20.          20.          20.
  20.          20.          20.          20.          20.
  20.          20.          20.          20.          20.
  20.          20.          20.          20.          20.
  20.          20.          20.          20.          20.
  20.          20.          20.          20.          20.
  20.          20.          20.          20.          20.
  20.         

Finally, use the functions above to calculate the temperature distribution as a function of time, and display this graphically using an appropriate plotting routine.

In [23]:
timesteps = 1000
f_u = np.zeros((timesteps, segments - 2))

print(f_u)
print(biggrids.timetravel())

for i in range(0,timesteps):
    f_u[i,:] = biggrids.timetravel()

print(f_u)

xr = np.arange(0,segments - 2,1)
yr = np.arange(0,timesteps,1)

X,Y = np.meshgrid(xr,yr)
X = X/(np.amax(X)) * x
print(X)
print(Y)

fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
ax.plot_surface(X, Y, f_u, vmin=f_u.min() * 2, cmap=cm.hot)


plt.show()


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[ 13.73666971  18.03853468  19.38573474  19.80763269  19.939757
  19.98113391  19.99409177  19.99814974  19.99942056  19.99981854
  19.99994317  19.9999822   19.99999443  19.99999825  19.99999945
  19.99999983  19.99999995  19.99999998  19.99999999  20.
  20.          20.          20.          20.          20.
  20.          20.          20.          20.          20.
  20.          20.          20.          20.          20.
  20.          20.          20.          20.          20.
  20.          20.          20.          20.          20.
  20.          20.          20.          20.          20.
  20.          20.          20.          20.          20.
  20.          20.          20.          20.          20.
  20.          20.          20.          20.          20.
  20.          20.          20.          20.          20.
  20.     

## Part 2 - Neumann Boundary Conditions

Now we assume the far end of the poker from the furnace is no longer held at 0 C, but instead experiences no heat loss. Again your task is to find the tempeterature distribution as a function of time.

In this case, you will need to implement a Neumann boundary condition at the end of the poker, to ensure the derivative $\frac{\partial u}{\partial x}$ is zero. Since we are using finite differences, this is equivalent to ensuring the final two noces have the same value.

The finite difference equation for node $i=(N-1)$ is :

$$u^n_{N-1} = -\alpha u^{n+1}_{N-2} + (1 + 2\alpha)u^{n+1}_{N-1} - \alpha u^{n+1}_{N}$$

To enforce the Neumann boundary condition we can substitute $u^{n+1}_{N} = u^{n+1}_{N-1}$, giving :

$$u^n_{N-1} = -\alpha u^{n+1}_{N-2} + (1 + \alpha)u^{n+1}_{N-1}$$

This results in a modified form of $M$, shown here for the example $N_x=7$, and the matrix ix $5\times5$ :  

$$M = \begin{pmatrix}
1+2\alpha & -\alpha   &           &           &           & \\
-\alpha   & 1+2\alpha & -\alpha   &           &           & \\
          & -\alpha   & 1+2\alpha & -\alpha   &           & \\
          &           & -\alpha   & 1+2\alpha & -\alpha   & \\
          &           &           & -\alpha   & 1+\alpha & \\
\end{pmatrix}$$

Note that you will also need to include a boundary term vector $b$, since the end of the poker in the furnace still requires a Dirichlet condition.

First write any new functions you need. You should be able to re-use some functions from Part 1.

Finally, use the functions above to calculate the temperature distribution as a function of time, and display this graphically using a sensible plotting function.

# Part 3

In the Markdown cell below, describe how your code solves the problem set. State any equations used in the solution and how they were obtained. Include your reasons for any libraries you used, as well as any particular programming techniques. Explain your choice of any test cases. Finally, state any salient features of the results you obtained. You are not expected to write more than about 250-300 words.